In [1]:
from xautoml.util.datasets import openml_task

X_train, y_train = openml_task(31, 0, train=True)
X_train

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xe. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,>=200,18,existing paid,furniture/equipment,3049.0,<100,<1,1,female div/dep/mar,none,1,life insurance,45,stores,own,1,unskilled resident,1,none,yes
1,no checking,24,critical/other existing credit,education,1927.0,no known savings,1<=X<4,3,female div/dep/mar,none,2,car,33,none,own,2,skilled,1,yes,yes
2,no checking,12,existing paid,new car,640.0,<100,1<=X<4,4,male div/sep,none,2,real estate,49,none,own,1,unskilled resident,1,none,yes
3,0<=X<200,6,existing paid,radio/tv,2063.0,<100,<1,4,male mar/wid,none,3,car,30,none,rent,1,high qualif/self emp/mgmt,1,yes,yes
4,<0,12,critical/other existing credit,used car,1526.0,<100,>=7,4,male single,none,4,no known property,66,none,for free,2,high qualif/self emp/mgmt,1,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0<=X<200,30,critical/other existing credit,new car,4249.0,<100,unemployed,4,male mar/wid,none,2,car,28,none,own,2,high qualif/self emp/mgmt,1,none,yes
896,0<=X<200,24,critical/other existing credit,radio/tv,1216.0,100<=X<500,<1,4,male single,none,4,no known property,38,bank,own,2,skilled,2,none,yes
897,<0,36,existing paid,furniture/equipment,2712.0,<100,>=7,2,male single,none,2,life insurance,41,bank,own,1,skilled,2,none,yes
898,<0,24,all paid,furniture/equipment,3552.0,<100,4<=X<7,3,male single,none,4,car,27,bank,own,1,skilled,1,none,yes


In [14]:
import numpy as np
import optuna
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm
from optuna import Trial
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

# Przechowuj modele z triali
models = {}

def objective(trial: Trial):
    # Definicja hiperparametrów do optymalizacji
    hyperparameters = {
        'max_features': trial.suggest_float('max_features', 0, 1),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy'])
    }

    cat_columns = make_column_selector(dtype_exclude=np.number)(X_train)
    num_columns = make_column_selector(dtype_include=np.number)(X_train)

    # Pipeline bez SelectKBest
    pipeline = Pipeline(steps=[
        ('enc', ColumnTransformer([
            ('ordinal', OrdinalEncoder(), cat_columns),
            ('scaler', MinMaxScaler(), num_columns)
        ])),
        ('clf', RandomForestClassifier(**hyperparameters)) 
    ])
    # pipeline.set_params(**hyperparameters)

    # Użycie cross-validation dla optymalizacji
    score = sklearn.model_selection.cross_val_score(pipeline, X_train, y_train, cv=3)
    accuracy = score.mean()

    # Zapisanie modelu w models
    models[trial.number] = pipeline.fit(X_train, y_train)

    return accuracy

# Tworzymy study i optymalizujemy
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.optimize(objective, n_trials=10)


[I 2025-04-11 09:15:27,729] A new study created in memory with name: no-name-a0cb92a5-41cd-4adc-b5f7-5acfbcbac069
[I 2025-04-11 09:15:28,073] Trial 0 finished with value: 0.7466666666666667 and parameters: {'max_features': 0.061131089254650495, 'criterion': 'gini'}. Best is trial 0 with value: 0.7466666666666667.
[I 2025-04-11 09:15:28,555] Trial 1 finished with value: 0.7466666666666667 and parameters: {'max_features': 0.450002062539248, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7466666666666667.
[I 2025-04-11 09:15:29,053] Trial 2 finished with value: 0.7433333333333333 and parameters: {'max_features': 0.5577139492228452, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7466666666666667.
[I 2025-04-11 09:15:29,595] Trial 3 finished with value: 0.7455555555555556 and parameters: {'max_features': 0.6530470787186425, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7466666666666667.
[I 2025-04-11 09:15:29,979] Trial 4 finished with value: 0.7477777777777778 and 

In [15]:
from xautoml.main import XAutoML
from xautoml.adapter import import_optuna
from xautoml.util.datasets import openml_task

X_test, y_test = openml_task(31, 0, test=True)

rh = import_optuna(study, models, metric='accuracy')
main = XAutoML(rh, X_test, y_test)
main

ValueError: Unknown hyperparameter max_features